In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from egoaxis import rot_axis_90deg_cc

## Module

In [ ]:
# Note:
# To make it match with the previous measurement, init with the last point
# of the previous measurement and keep it fixed.
# Use the largest principal component vector, project points, keep last point
# fixed an rotate line around this point, so the first point matches the
# reference point from the last measurement.

class Polyline(object):
    def __init__(self, vertices=None):
        if vertices is None:
            self._vertices = np.empty((0, 2))
            self._segment_lengths = np.empty(0)
        else:
            vertices = np.atleast_2d(vertices)
            if len(vertices.shape) != 2 or vertices.shape[1] != 2:
                raise ValueError("vertices must have shape (N, 2)")
            
            if len(self._vertices) > 1:
                self._segment_lengths = [
                    self._get_segment_length(v0, v1) for v0, v1 in
                    zip(self._vertices[:-1], self._vertices[1:])
                ]

    def get_value(self, t):
        """
        Return the (interpolated) value at the given parameter t (in [0, 1])
        """
        t = np.clip(np.asarray(t), 0., 1.)
        # Select segments to interpolate each t in
        cumlen = (np.cumsum(self._segment_lengths) /
                  np.sum(self._segment_lengths))
        cumlen = np.insert(cumlen, 0, 0.)
        segment_idx = np.searchsorted(cumlen, t)
        m = segment_idx > 0
        v0 = np.repeat(self._vertices[[0]], repeats=self.nvertices, axis=0)
        v1 = np.repeat(self._vertices[[1]], repeats=self.nvertices, axis=0)
        v0[m] = self._vertices[segment_idx - 1][m]
        v1[m] = self._vertices[segment_idx][m]
        
#         if segment_idx > 0:
#             v0 = self._vertices[segment_idx - 1]
#             v1 = self._vertices[segment_idx]
#         else:
#             v0 = self._vertices[0]
#             v1 = self._vertices[1]
            
        # Normalize ts in [0, 1] relative to corresponding segment
        segment_idx = np.maximum(0., segment_idx - 1)
        t_norm = ((t - cumlen[segment_idx]) /
                (cumlen[segment_idx + 1] - cumlen[segment_idx]))

        # Interpolated x and y values at ts
        v = (v1 - v0) * t_norm + v0
        # x = (v1[:, 0] - v0[:, 0]) * t_norm + v0[:, 0]
        # y = (v1[:, 1] - v0[:, 1]) * t_norm + v0[:, 1]
        return x, y
    
    def get_dist_to_line(self, pt):
        """
        Returns the smallest distance to any line segment in the polyline.
        """
        return
    
    def append_vertex(self, v):
        """ Append a vertex v to the polyline """
        self.insert_vertex(v, self.nvertices)
        
    def insert_vertex(self, v, i):
        """ Insert a vertex v to the polyline at position i """
        v = np.atleast_2d(v)
        self._vertices = np.insert(self._vertices, i, v)
        _v = self._vertices
        if len(_v) > 1:
            if i == 0:
                l = self._get_segment_length(_v[0], _v[1])
            else:
                l = self._get_segment_length(_v[i], _v[i-1])                
            self._segment_lengths.insert(i, l)
            
    def clear(self):
        self._vertices = np.empty((0, 2))
        self._segment_lengths = np.empty(0)
        
    def _get_segment_length(self, v0, v1):
        return np.sqrt((v1[:, 0] - v2[:, 0])**2 + (v1[:, 1] - v2[:, 1])**2)
    
    def _get_dist_to_segment(self, pt, idx):
        """
        Get distance of pt to specified segment in the polyline.
        """
        v0 = self._vertices[idx]
        v1 = self._vertices[idx + 1]
        
        # Project pt on line
        
        return
     
    @property
    def nvertices(self):
        return len(self._vertices)
    
    @property
    def nsegments(self):
        return len(self._segment_lengths)
    
    @property
    def vertices(self):
        return self._vertices.copy()

    @property
    def segment_lengths(self):
        return self._segment_lengths.copy()
    
    @property
    def line_length(self):
        return np.sum(self._segment_lengths)
    
    @property
    def xcoords(self):
        return self._vertices[:, 0]

    @property
    def ycoords(self):
        return self._vertices[:, 1]

        
def proj_points(pts, vec):
    """
    Projects points with shape (len(vec), N) along direction of vec (1d-array).
    """
    vec = np.asarray(vec)
    pts = np.atleast_2d(pts)
    
    unit = vec / np.linalg.norm(vec)
    proj_len = np.matmul(pts.T, vec)

    return proj_len * vec.reshape(len(vec), 1), proj_len


def get_max_eigvec(data):
    # Get direction and value of largest eigenvalue from cov(data)
    cov = np.cov(data)
    eigvals, eigvecs = np.linalg.eigh(cov)
    eigvec_max = eigvecs[np.argmax(eigvals)]
    return eigvec_max, eigvals, eigvecs


def poly_principal_curve(data, niter="auto"):
    """
    Computes a principal curve for the given data points (shape (2, n)).
    Source
    ------
    Kegl: Learning and Design of Principal Curves
        (https://www.lri.fr/~kegl/research/PDFs/KeKrLiZe00.pdf)
    """
    data = np.atleast_2d(data)
    if len(data.shape) != 2 or data.shape[0] != 2:
        raise ValueError("'data' must have shape (2, nPoints).")
    N = data.shape[1]
    
    # Center data at  (0, 0)
    mean = np.mean(data, axis=1).reshape(2, 1)
    data -= mean
    
    # Following the algorithm in chapter 3 of the paper
    
    # Initialization: Use first principal value of data as projection line d and
    # pick the line length L as the distance of min, max points projected on d
    eigvec_max, _, _ = get_max_eigvec(data)
    # Project along largest eigenvector and determine line length by finding
    # the outermost projected points along the projection line
    proj_vec, proj_len = proj_points(data, eigvec_max)
    idx = np.argsort(proj_len)
    # Keep the original order intact for the projection
    idx_min, idx_max = ((idx[0], idx[-1]) if
                        idx[0] < idx[-1] else (idx[-1], idx[0]))
    dmin, dmax = data[:, idx_min], data[:, idx_max]
    length = np.linalg.norm(dmax - dmin)
    
    # Iteration: Add vertex each iter until converged or max vertices reached
    pl = Polyline([dmin, dmax])
    converged = False
    # Empirical from paper, at least 2
    max_vertices = max(int(N**(1. / 3.)) + 1, 2.)
    for i in range(max_vertices):
        # Vertex optimization:
        # 1. Optimize each existing vertex while fixing the others
        # 2. As long as the curve with k current indices is locally closest
        step_converged = False
        while not step_converged:
            for j, vert in pl.vertices:
                pass
            step_converged = True
    
    return dmin, dmax, length

In [ ]:
pl = Polyline()

In [ ]:
pl.append_vertex([1, 2])

In [ ]:
dmin, dmax, _ = poly_principal_curve(data)

In [ ]:
curve = Curve2D(curvature=-4., curve_rel_start=0.3)
x = np.linspace(0, 10, 50)
y, dy = curve(x)
x_sam = np.linspace(1, 10, 20)
x_sam, y_sam = curve.sample(x_sam, stddev=0.4, seed=2)

data = np.asarray([x_sam, y_sam])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Truth and sampled points
ax.plot(x, y)
ax.plot(x_sam, y_sam, marker="o", ls="-")

ax.set_xlim(0, 1.1 * max(np.amax(x_sam), np.amax(x)))
ax.set_ylim(-5, 5)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.plot(0, 0, marker=10, ms=10, ls="none", c="k")

# Principal value line
eigvec, _, _ = get_max_eigvec(data)
mean = np.mean(data, axis=1)
mx = np.r_[mean[0] - 10 * eigvec[0], mean[0] + 10 * eigvec[0]]
my = np.r_[mean[1] - 10 * eigvec[1], mean[1] + 10 * eigvec[1]]
ax.plot(mean[0], mean[1], ls="none", marker="x", ms=10, c="C3", mew=3, zorder=1)
ax.plot(mx, my, ls="--", c="C3", lw=2)

# Projected points
mean = np.mean(data, axis=1).reshape(2, 1)
proj_vec, proj_len = proj_points(data - mean, eigvec)
proj_vec += mean
for proj, pt in zip(proj_vec.T, data.T):
    dx = [proj[0], pt[0]]
    dy = [proj[1], pt[1]]
    ax.plot(dx, dy, lw=1, c="k")

# ax = rot_axis_90deg_cc(ax)
ax.grid()
ax.axvline(0, 0, 1, c="k", lw=1, zorder=1)
ax.axhline(0, 0, 1, c="k", lw=1, zorder=1)
ax.set_axisbelow(True)
ax.set_aspect("equal")

plt.show()

## Generator

In [ ]:
# Flaws: Not a real parametric curve, densitiy of sampled points change
#        around line when derivative get larger
class Curve2D(object):
    def __init__(self, curvature=0, curve_rel_start=0):
        """
        curvature = scaling factor of first quarter of sine wave
        curve_rel_start in [0, 1] -> 0 = curve start at beginning, 1 = at end 
        """
        if curve_rel_start < 0 or curve_rel_start > 1:
            raise ValueError("curve_rel_start must be in [0, 1]")
        self.curvature = curvature
        self.curve_rel_start = curve_rel_start

    def sample(self, x, stddev=1., seed=None):
        """
        Model: (x, y) = (x, f(x)) + rot(alpha) * norm(0, stddev)
        where rot(alpha) rotates the point to the normal vector of the curve at
        f(x) (samples with stddev along the normal vector at each (x, f(x))).
        """
        x = np.asarray(x)
        y, dy = self(x)
        # Sample offset along normal vector
        rndgen = np.random.RandomState(seed)
        # y_sam = rndgen.normal(0, stddev, size=x.shape)
        y_sam = rndgen.uniform(-stddev, stddev, size=x.shape)
        y_sam = rndgen.normal(0, stddev, size=x.shape)
        x_sam = np.zeros_like(x)
        # Rotate sampled points along normal vector
        dx = np.ones_like(x)
        angles = np.arctan2(dy, dx)
        s, c = np.sin(angles), np.cos(angles)
        rot = np.asarray([ [[ci, -si], [si, ci]] for si, ci in zip(s, c) ])
        vec = np.asarray([
            [xi, yi] for xi, yi in zip(x_sam, y_sam) ])
        rot_vec = np.asarray([
            np.matmul(roti, veci) for roti, veci in zip(rot, vec) ])
        
        # print("x, y", x, y)
        # print("dx, dy", dx, dy)
        # print("angles", np.rad2deg(angles))
        # print("vecs", vec)
        # print("rotmats", rot)
        # print("rot vecs", rot_vec)
        
        return x + rot_vec[:, 0], y + rot_vec[:, 1]
    
    def __call__(self, x):
        """
        Get y values of the curve for given x
        """
        x = np.asarray(x)
        # Quarter sine period for the curvature starting at curve_rel_start
        xmax = np.amax(x)
        xmin = self.curve_rel_start * xmax
        normed = np.zeros_like(x)
        m = x > xmax * self.curve_rel_start
        # Transform to [pi / 2, pi]
        normed[m] = (x[m] - xmin) / (xmax - xmin) * np.pi / 2. + np.pi / 2.
        shift = np.zeros_like(x)
        shift[m] = 1.
        func = self.curvature * (np.sin(normed) - shift)
        # Get gradient (note: respect inner derivative from normalization)
        grad = np.zeros_like(x)
        grad[m] = (self.curvature * np.cos(normed[m]) /
                   (xmax - xmin) * np.pi / 2.)
        return func, grad

## Main

In [ ]:
curve = Curve2D(curvature=-3., curve_rel_start=0.6)
x = np.linspace(0, 10, 50)
y, dy = curve(x)
x_sam = np.linspace(1, 10, 20)
x_sam, y_sam = curve.sample(x_sam, stddev=0.4, seed=2)

fig, ax = plt.subplots(1, 1)
ax.plot(x, y)
ax.plot(x_sam, y_sam, marker="o", ls="-")

ax.set_xlim(0, 1.1 * max(np.amax(x_sam), np.amax(x)))
ax.set_ylim(-5, 5)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.plot(0, 0, marker=10, ms=10, ls="none", c="k")

ax = rot_axis_90deg_cc(ax)
ax.grid()
ax.axvline(0, 0, 1, c="k", lw=1, zorder=1)
ax.set_axisbelow(True)

plt.show()